# AB Model from spec

## Setup

Package installation

In [ ]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [1]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
from datetime import datetime
import logging
from collections import Counter
import networkx as nx

# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [2]:
import EV.model_config as cfg
import EV.worker as worker
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachines import EVSM, LSM
from EV.modelquery import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered

## Model Environment testing

Station config import

In [3]:
station_config['A-B']['CS_AB_1']
# worker.sum_total_charging_stations(station_config)

NameError: name 'station_config' is not defined

In [ ]:
# {'A-B': {'CS_A-B_1': [{'CPID': 'CS_A-B_1_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_1_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_1_3',
#     'Power': '60',
#     'Distance': '40',
#     'Price': '10',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_2': [{'CPID': 'CS_A-B_2_1',
#     'Power': '7',
#     'Distance': '20',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_2_2',
#     'Power': '7',
#     'Distance': '20',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_3': [{'CPID': 'CS_A-B_3_1',
#     'Power': '7',
#     'Distance': '50',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_3_2',
#     'Power': '7',
#     'Distance': '50',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_3_3',
#     'Power': '60',
#     'Distance': '50',
#     'Price': '10',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_4': [{'CPID': 'CS_A-B_4_1',
#     'Power': '7',
#     'Distance': '10',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_4_2',
#     'Power': '7',
#     'Distance': '10',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_4_3',
#     'Power': '60',
#     'Distance': '10',
#     'Price': '10',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-B_5': [{'CPID': 'CS_A-B_5_1',
#     'Power': '7',
#     'Distance': '30',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_5_2',
#     'Power': '7',
#     'Distance': '30',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-B_5_3',
#     'Power': '7',
#     'Distance': '30',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}]},
#  'A-C': {'CS_A-C_1': [{'CPID': 'CS_A-C_1_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_1_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-C_2': [{'CPID': 'CS_A-C_2_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_2_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-C_3': [{'CPID': 'CS_A-C_3_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_3_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}],
#   'CS_A-C_4': [{'CPID': 'CS_A-C_4_1',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'},
#    {'CPID': 'CS_A-C_4_2',
#     'Power': '7',
#     'Distance': '40',
#     'Price': '6',
#     'Green': '0',
#     'Booking': '0'}]}}


In [ ]:
# worker.get_routes(station_config)                   #works
worker.find_cpid_for_charging_station(station_config, 'CS_A-C_1')                 # doesnt work, anymore due to new structure

In [ ]:
# worker.count_charging_stations('A-B', station_config)



# worker.get_target_charging_stations('A-B', station_config)                   # Works
# worker.sum_total_charging_stations(station_config)                           # Works   
# worker.get_routes(station_config)                                            # Works
# worker.total_route_length(station_config, 'A-B')                             # Works
# worker.cpids_for_route(station_config, 'A-B') # not working

# worker.get_checkpoint_list(station_config, 'A-C')                           # Works
# worker.get_charging_stations_on_route(station_config, 'A-C')                    # Works
# worker.charge_points_on_route(station_config, 'A-B')                    # not working
# worker.get_route_cps(station_config, 'A-B')                              #not working snymore

# worker.find_cpid_for_charging_station(station_config, 'CS_AB_1_1')                    # not works



In [ ]:
a = worker.get_routes(station_config)
print(len(a))  
print(a)

In [ ]:
# worker.get_checkpoint_list(station_config, 'A-B')
# worker.charge_points_on_route('A-B', station_config)
# worker.get_charge_points_per_station('A-B', station_config)

In [ ]:
station_config = worker.read_charging_data(cfg.STATION_PATH +'stations.csv')

In [ ]:
worker.num_stations_per_route(station_config)

In [ ]:
# station_config   

In [ ]:
# worker.get_stations_for_route('A-B', station_config)
# stations = worker.get_stations_for_route('A-B', station_config)
# print(stations)  # Output: ['CS_A-B_1', 'CS_A-B_2', 'CS_A-B_3', 'CS_A-B_4', 'CS_A-B_5']


From input matrix to model params

Approach 2

In [ ]:
station_config = worker.read_charging_stations(cfg.STATION_PATH +'stations.csv')

In [ ]:
route_counts = Counter([station['Route'] for station in station_config])
print(route_counts)

In [ ]:
points_per_station_per_route = {}
for station in station_config:
    key = f"{station['Route']}_{station['Station']}"
    if key not in points_per_station_per_route:
        points_per_station_per_route[key] = []
    points_per_station_per_route[key].append(station['CPID'])
# print(points_per_station_per_route)
print(points_per_station_per_route.keys())

In [ ]:
routes = list(set([station['Route'] for station in station_config]))
print(routes)

In [ ]:
num_routes = len(routes)
print(num_routes)

## Valid Model Environment

Approach 3

In [8]:
# station_config = worker.read_csv(cfg.STATION_PATH +'stations.csv')
station_config = cfg.station_config
station_config


In [26]:
# get the number of charging stations per route
stations_per_route = {route: len(station_config[route]) for route in station_config}
print(stations_per_route)

{'A-B': 1, 'A-C': 2, 'A-D': 2, 'B-C': 2, 'B-D': 1, 'C-D': 1}


In [27]:
# get the number of charging points per station per route
points_per_station_per_route = {}
for route in station_config:
    for station in station_config[route]:
        num_points = len(station_config[route][station])
        if route not in points_per_station_per_route:
            points_per_station_per_route[route] = {}
        points_per_station_per_route[route][station] = num_points
print(points_per_station_per_route)

{'A-B': {'CS_A-B_1': 2}, 'A-C': {'CS_A-C_1': 3, 'CS_A-C_2': 2}, 'A-D': {'CS_A-D_1': 3, 'CS_A-D_2': 3}, 'B-C': {'CS_B-C_1': 2, 'CS_B-C_2': 3}, 'B-D': {'CS_B-D_1': 1}, 'C-D': {'CS_C-D_1': 2}}


In [28]:
# get a list of all routes
routes = station_config.keys()
print(routes)

# get the total number of routes
num_routes = len(routes)
print(num_routes)

dict_keys(['A-B', 'A-C', 'A-D', 'B-C', 'B-D', 'C-D'])
6


In [29]:
# worker.get_charge_points_per_station_on_route(station_config, 'A-B')
cs = worker.count_charge_points_by_station(station_config, 'A-B')                        #works
# type(cs)
# b = cs.values()
# b = list(b)
# type(b)
cs

{'CS_A-B_1': 2}

In [29]:
a = worker.get_charging_stations_on_route(station_config, 'A-C') 
print(a)

['CS_A-C_1', 'CS_A-C_2']


In [30]:
cs = worker.get_route_from_config('A-C', station_config)  
print(cs)

{'40': [{'CPID': 'CS_AC_1_1', 'Power': '22', 'Distance': '40', 'Price': '70.4', 'Green': '10', 'Booking': '0'}, {'CPID': 'CS_AC_1_2', 'Power': '22', 'Distance': '40', 'Price': '70.4', 'Green': '10', 'Booking': '0'}, {'CPID': 'CS_AC_1_3', 'Power': '50', 'Distance': '40', 'Price': '70.4', 'Green': '10', 'Booking': '0'}, {'CPID': 'CS_AC_2_1', 'Power': '22', 'Distance': '40', 'Price': '70.4', 'Green': '10', 'Booking': '0'}, {'CPID': 'CS_AC_2_2', 'Power': '22', 'Distance': '40', 'Price': '70.4', 'Green': '10', 'Booking': '0'}]}


In [ ]:
def cumulative_cs_distances(numbers):
    result = []
    for i in range(len(numbers)):
        if i == 0:
            result.append(numbers[i])
        else:
            result.append(numbers[i] + result[i-1])
    return result

def get_values(d):
    """
    Returns a list containing the associated values for each key in the dictionary.
    """
    return list(d.values())


In [ ]:
# worker.get_cumulative_distances('A-B', station_config)
list = ['a', 'b', 'c', 'd', 'e']

In [ ]:
a = worker.remove_list_item_seq(list)
print(a)
b = worker.remove_list_item_seq(list)
print(b)
c = worker.remove_list_item_seq(list)
print(c)
d = worker.remove_list_item_seq(list)
print(d)
e = worker.remove_list_item_seq(list)
print(e)
print(len(list))

In [ ]:
def select_route_as_key(input):
    """
    This function returns one of the keys in input dictionary, up to the integer value of the key
    """
    # calculate the sum of values in the input dictionary
    n = sum(input.values())
    
    # create a dictionary to keep track of the number of times each key is returned
    counter = {key: 0 for key in input}
    
    # select a key and return it as a string
    def helper():
        for key in input:
            if counter[key] < input[key]:
                counter[key] += 1
                return str(key)
        # if all keys have been returned the maximum number of times, raise an exception
        raise Exception('no more route assignments')
    
    # keep track of the number of times the function has been run
    num_runs = 0
    
    # run the function at most n times
    while num_runs < n:
        try:
            key = helper()
            num_runs += 1
            yield key
        except Exception as e:
            yield str(e)
            return


In [ ]:
input_dict = {'key1': 4, 'key2': 1, 'key3': 1}
for key in select_route_as_key(input_dict):
    print(key)

In [14]:
def get_charging_stations_along_route(station_config, route_name):
    """
    Returns a dictionary of charging stations along the route. 
    The key is the station name and the value is the distance from the start of the route.   
    """
    charging_stations = {}
    route_stations = station_config.get(route_name)
    if route_stations:
        for station_name, station_data in route_stations.items():
            for station in station_data:
                charging_stations[station_name] = int(station['Distance'])
    return charging_stations


In [15]:
# a = get_charging_stations_along_route(station_config, 'A-B')
# print(a)
b = worker.get_dict_values(worker.get_charging_stations_along_route(station_config, 'A-B'))
print(b)
c = worker.cumulative_cs_distances(b)
print(c)


NameError: name 'station_config' is not defined

In [31]:

def get_power_values(station_config, route_name):
    """Returns a list of all distance values, for every CP on a given route."""
    distance_values = []
    for station in station_config[route_name]:
        for charger in station_config[route_name][station]:
            distance_values.append(int(charger['Power']))
    return distance_values

In [32]:
a = get_power_values(station_config, 'A-B')
print(a)

[22, 7]


In [5]:
def get_power_values_route(station_config, route_name):
    """Returns a dict with Charge station name as key and list of power values for each charge point in the station, for a given route."""
    power_dict = {}
    for station in station_config[route_name]:
        cp_power = []
        for charger in station_config[route_name][station]:
            cp_power.append(int(charger['Power']))
        power_dict[station] = cp_power
    return power_dict


In [47]:
# b = worker.get_power_value_for_cp(station_config, 'B-C', 'B-C_2', 2)
# print(b)

None


In [10]:
a = worker.get_power_values_route(station_config, 'B-C')
print(a)

{'CS_B-C_1': [50, 50], 'CS_B-C_2': [22, 22, 50]}


In [59]:
# def get_value(dictionary, input_string, index):
#     for key in dictionary:
#         if input_string in key:
#             return dictionary[key][index]
#     return None

def get_cp_value(dictionary, input_string, index):
    for key in dictionary:
        if input_string in key:
            return dictionary[key][index - 1]
    return None


In [ ]:
# get_value(a, 'CS_B-C_1', 1)

In [25]:
worker.get_cp_value(a, 'CS_B-C_2', worker.cp_name_to_cp_number('cp_1'))

22

In [55]:
worker.cp_name_to_cp_number('cp_1')

1

In [37]:
def get_string_after_hyphen(string):
    """
    Given a string that contains a hyphen, returns the part of the string after the hyphen.
    If the string does not contain a hyphen, returns an empty string.
    """
    if '-' in string:
        return string.split('-')[1]
    else:
        return ''
a = get_string_after_hyphen('Lagos-Abuja')
print(a)

Abuja


Model parameters

In [ ]:
class Object(object):
    def __init__(self):
        self.post_1 = 0
        self.post_2 = 3
        self.post_3 = 9
        self.points = [20,40,60]

def get_value_by_index(obj, index):
    post_attrs = sorted([key for key in vars(obj) if key.startswith('post_')], key=lambda x: int(x.split('_')[1]))
    post_key = post_attrs[index-1]
    post_value = getattr(obj, post_key)
    points_value = obj.points[index-1]
    return post_value * points_value

obj = Object()
value = get_value_by_index(obj, 3) # returns 540
print(value)

In [ ]:
# 365 days = 8760 hours, 30 days = 720 hours, 2 days = 48 hours, 7 days = 168 hours
ticks = 48
no_evs = 2

In [38]:
def reverse_strings(string_list):
    reversed_list = []
    for string in string_list:
        reversed_list.append(string[::-1])
    return reversed_list

In [39]:
a = ['A-B', 'B-C', 'A-C']
b = worker.reverse_strings(a)
for i in b:
    a.append(i)
print(a)

['A-B', 'B-C', 'A-C', 'B-A', 'C-B', 'C-A']


In [40]:
def get_possible_journeys(current_location, permutation):
    cities = ['A', 'B', 'C', 'D']
    journeys = []
    
    for p in permutation:
        if p == current_location:
            continue
        elif p in cities:
            journeys.append(current_location + '-' + p)
        else:
            for c in cities:
                if c != current_location:
                    journeys.append(current_location + '-' + c + '-' + p)
    
    return journeys

def get_possible_destinations(current_position):
    cities = ['A', 'B', 'C', 'D']
    destinations = []
    
    for city in cities:
        if city == current_position:
            continue
        else:
            destinations.append(city)
    
    return destinations

def get_possible_journeys_long(current_position):
    cities = ['A', 'B', 'C', 'D']
    journeys = []
    
    for city in cities:
        if city == current_position:
            continue
        else:
            journeys.append(current_position + '-' + city)
    
    return journeys



In [41]:
# get_possible_journeys('A', 'B')
get_possible_destinations('A')
# get_possible_journeys_long('D')

['B', 'C', 'D']

NetworkX

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.animation as animation
import matplotlib
import numpy as np

matplotlib.use('TkAgg')
plt.ion()

H = nx.octahedral_graph()  # generate a random graph
pos = nx.spring_layout(H, iterations=200)  # find good positions for nodes

In [ ]:
# nodes = list(range(100))

# df = pd.DataFrame({'from': np.random.choice(nodes, 100),
#                    'to': np.random.choice(nodes,100)
#                   })

nodes = ['A', 'B', 'C', 'D']

df = pd.DataFrame({'from': np.random.choice(nodes, len(nodes)),
                   'to': np.random.choice(nodes,len(nodes))
                  })

In [ ]:
for i in range(len(nodes)):
    df['from'][i] = nodes[i]
    df['to'][i] = nodes[i]
# get_possible_destinations('A')

In [ ]:
G = nx.from_pandas_edgelist(df, source='from', target='to')
nx.draw(G);
plt.hist([v for k,v in nx.degree(G)]);

In [ ]:
import networkx as nx

def create_charging_network(station_config):
    G = nx.Graph()

    # Add city nodes to graph
    cities = set()
    for route, config in station_config.items():
        cities.update([route.split('-')[0], route.split('-')[1]])
    for city in cities:
        G.add_node(city, node_type='city')

    # Add charging station nodes and edges to graph
    for route, config in station_config.items():
        start_city = route.split('-')[0]
        end_city = route.split('-')[1]

        # Add start and end city nodes to graph
        G.add_edge(start_city, end_city, edge_type='route')

        # Add charging station nodes to graph
        for station_id in config:
            G.add_node(station_id, node_type='station')
            G.add_edge(start_city, station_id)
            G.add_edge(station_id, end_city)

    return G

G = create_charging_network(station_config)


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# assume the returned graph object is stored in variable G
nx.draw(G, with_labels=True)
plt.show()


Run Model

In [ ]:
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, params=station_config)
for i in range(ticks):
    model_run.step()

In [ ]:
# model_run.schedule.agents
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

Export results to CSV file

In [ ]:
# today's date as string
date_str = str(datetime.today())
# export csv
model_run.datacollector.get_model_vars_dataframe().to_csv(cfg.DATA_PATH + 'data_' + date_str[0:10] + '_' + str(no_evs) + '_EV_agent_model_output.csv')

Try 3 - unimplemented

In this implementation, the move method first determines the position of the next ChargeStation agent in the route, and then moves the EV agent towards that position one step at a time. At each step, the method checks if there is a ChargeStation agent at the new position, and if there is one, it runs the drop_by method to append the ChargeStation agent to the chosen_station attribute of the EV agent.

Note that the move method assumes that the pos attribute of the EV agent is a tuple of coordinates, and that the positions of the Location and ChargeStation agents are also tuples of coordinates. The method also assumes that the grid used in the model is a MultiGrid object. If any of these assumptions are not true in your implementation, you may need to modify the code accordingly.

This is a method for an agent class that represents an electric vehicle (EV) in a simulation. The method is called "move" and it takes a "model" argument which represents the current state of the simulation.

The method begins by calculating a unit vector towards the EV's destination, which is stored in the "dest_pos" attribute of the agent. The distance between the current position of the EV and the destination is also calculated using the Pythagorean theorem.

If the distance is 0, the EV has reached its destination and the "dest_pos" attribute is set to None.

If the distance is not 0, the vector is normalized to get a unit vector that represents the direction the EV should move in. The next position of the EV is then calculated by adding the unit vector to the current position of the agent.

The method checks if the next position is within the boundaries of the grid. If it is, the EV is moved to the next position by calling the "move_agent" method of the model's grid attribute. The method then checks if the EV has encountered a ChargeStation agent at its new position. If it has, the EV is added to the ChargeStation's queue by appending itself to the queue attribute of the ChargeStation agent.

If the next position is outside the grid boundaries, the "dest_pos" attribute of the EV is set to None, indicating that the EV has reached the edge of the grid and can no longer move.

## Data import and preprocessing

Data import and Helper functions

In [ ]:
import analysis.helper as helper
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

In [ ]:
data = pd.read_csv(cfg.DATA_PATH +'modeldatadata_2023-03-19_500_EV_agent_model_output.csv')

data = data.drop(columns=['Unnamed: 0'])

In [ ]:
# data.info()
data.head()

In [ ]:
a = helper.unpack_and_join(data, 'EVs Charge Level')
print(a)

In [ ]:
# # EV charge level per EV per timestep - 20 EVs unpacked
# newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
# newdf.head()

# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4']]
newdf.head()

In [4]:
from EV import model_config as cfg
data = cfg.station_config

In [5]:
def get_charge_points_per_station(route_name, station, data_dict):
    charge_points = []
    for key, value in data_dict.items():
        if value['Route'] == route_name:
            charge_points.append(sum([int(v['Power']) for k, v in data_dict.items() if v['Station'] == value['Station']]))
    return charge_points

def get_power_values_for_route(station_config, route_name):
    """Returns a dict with Charge station name as key and list of power values for each charge point in the station."""
    power_dict = {}
    for station in station_config[route_name]:
        cp_power = []
        for charger in station_config[route_name][station]:
            cp_power.append(int(charger['Power']))
        power_dict[station] = cp_power
    return power_dict

In [ ]:
data

In [6]:
# get_charge_points_per_station('A-B', data)
get_power_values_for_route(data, 'A-D')

{'CS_A-D_1': [22, 22, 50], 'CS_A-D_2': [22, 22, 50]}

In [7]:
b = {'CS_A-D_1': [22, 22, 50], 'CS_A-D_2': [22, 22, 50]}
c = []
pos = 0
for key, value in b.items():
    print(key, value)
    c.append(value[pos])
print(c)

# for a in self.__dir__():
#     print(a)

def get_power_value_for_cp(station_config, route_name, cs_name, cp_name):
    """Returns the power value for a specific charge point."""
    power_dict = get_power_values_for_route(station_config, route_name)
    for key, value in power_dict.items():
        if key == cs_name:
            return value[cp_name]

get_power_value_for_cp(data, 'A-D', 'CS_A-D_1', 2)

CS_A-D_1 [22, 22, 50]
CS_A-D_2 [22, 22, 50]
[22, 22]


50

In [8]:
# a = get_power_value_for_cp(data, 'B-C', 'cp_01')
# print(a)
# def cp_name_to_cp_number(cp_name):
#     """Returns the charge point number from the charge point name."""
#     return int(cp_name.split('p')[1])

# cp_name_to_cp_number('cp1')
# for i in range(1,5):
#     print(i)

1
2
3
4


In [9]:
print(worker.get_power_value_for_cp(station_config=data, route_name='A-D', cs_name='CS_A-D_1', cp_name = worker.cp_name_to_cp_number('cp1')))

IndexError: list index out of range

In [11]:
cp_name = worker.cp_name_to_cp_number('cp_1')
print(type(cp_name))

<class 'int'>


## Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

## Visualisations

Scatter plot

In [ ]:
# plot_data_lines(newdf, 'Timestep', ['EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'])

## Scrapbook

In [ ]:
import math

def distance(x1, y1, x2, y2)-> float:
        return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [ ]:
a = distance(1,2,4,4)
b = distance(1,3,2,2)
print(a,b)

In [ ]:
class MyListClass:
    def __init__(self, n, string_list):
        """
        # This function creates an ’n’ number of class attributes named as each string in the list, and initialised as empty lists.
        """
        for s in string_list:
            setattr(self, s, [])
            for i in range(n):
                getattr(self, s).append(None)

my_obj = MyListClass(5, ['foo', 'bar', 'baz'])
print(my_obj.foo)
# [None, None, None, None, None]

my_obj.foo[0] = 42
print(my_obj.foo)
# [42, None, None, None, None]

my_obj.bar.append('hello')
print(my_obj.bar)
# [None, None, None, None, None, 'hello']

In [ ]:
def get_direction(s):
    """Gets the direction of the EV based on the route.
    """
    # Extract the source and destination points from the input string
    src, dest = s.split('-')
    # Define a dictionary to map each point to its row and column in the grid
    point_map = {'A': (0, 0), 'B': (0, 1), 'C': (1, 0), 'D': (1, 1)}
    # Get the row and column numbers for the source and destination points
    src_row, src_col = point_map[src]
    dest_row, dest_col = point_map[dest]
    # Calculate the row and column differences between the source and destination points
    row_diff = dest_row - src_row
    col_diff = dest_col - src_col
    # Determine the direction based on the row and column differences
    if row_diff == -1 and col_diff == 0:
        return 1  # up
    elif row_diff == 0 and col_diff == 1:
        return 2  # right
    elif row_diff == 1 and col_diff == 0:
        return 3  # down
    elif row_diff == 0 and col_diff == -1:
        return 4  # left
    elif row_diff == -1 and col_diff == 1:
        return 5  # diagonal - right and up
    elif row_diff == -1 and col_diff == -1:
        return 6  # diagonal - left and up
    elif row_diff == 1 and col_diff == 1:
        return 7  # diagonal - right and down
    elif row_diff == 1 and col_diff == -1:
        return 8  # diagonal - left and down
    else:
        raise ValueError('Invalid input: {}'.format(s))
    
get_direction('B-C')